In [ ]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### TESTE DE CONEXÃO COM BANCO DE DADOS

In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import glob
import os

# Conectar ao PostgreSQL (NeonDB)
conn = psycopg2.connect(
    dbname="neondb",  # Substitua com o nome do seu banco de dados
    user="neondb_owner",  # Substitua com o nome de usuário
    password="npg_ayeAjRZ8sPD7",  # Substitua com sua senha
    host="ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech",  # Substitua com o host se necessário
    port="5432"  # Substitua com a porta se necessário
)

try:
    conn = psycopg2.connect(
        dbname="neondb",
        user="neondb_owner",  # Substitua com o nome de usuário
        password="npg_ayeAjRZ8sPD7",  # Substitua com sua senha
        host="ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech",  # Substitua com o host se necessário
        port="5432"
    )
    cur = conn.cursor()
    cur.execute("SELECT version();")
    versao = cur.fetchone()
    print(f"Conexão bem-sucedida! Versão do PostgreSQL: {versao[0]}")
    cur.close()
    conn.close()
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


Conexão bem-sucedida! Versão do PostgreSQL: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


#### GERA UM DATAFRAME COMBINANDO OS DADOS DOS ARQUIVOS DA PASTA INPUT

In [1]:
import pandas as pd
import os

# Caminho da pasta contendo os arquivos
folder_path = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/01-Itau/Input'

# Obter lista de arquivos na pasta com a extensão .xls ou .xlsx
arquivos = [f for f in os.listdir(folder_path) if f.endswith('.xls') or f.endswith('.xlsx')]

# Lista para armazenar os DataFrames processados
df_list = []

# Lista fixa com os nomes das colunas
nomes_colunas = [
    'data_lanc', 'lancamento', 'ag_origem', 'valor', 'saldos', 'idcategoria', 'observacao'
]

# Função para aplicar os nomes fixos nas colunas
def aplicar_nomes_colunas(df, novos_nomes):
    if len(novos_nomes) == len(df.columns):
        df.columns = novos_nomes
    else:
        print(f"Erro: A quantidade de novos nomes não corresponde ao número de colunas em {arquivo}.")
    return df

# Iterar sobre os arquivos da pasta
for arquivo in arquivos:
    # Caminho completo do arquivo
    file_path = os.path.join(folder_path, arquivo)

    # Ler o arquivo Excel (independente de ser .xls ou .xlsx)
    df = pd.read_excel(file_path, header=None)  # Não utiliza um cabeçalho por enquanto

    # Encontrar a linha que contém o cabeçalho (a que tem 'data' na primeira coluna)
    header_row = df[df.iloc[:, 0].str.contains('data', case=False, na=False)].index[0]

    # Ler novamente o arquivo, agora utilizando o cabeçalho correto
    df = pd.read_excel(file_path, header=header_row)

    # Aplicar os nomes fixos nas colunas
    df = aplicar_nomes_colunas(df, nomes_colunas)

    # Adicionar uma coluna com o nome do arquivo
    df['nome_arquivo'] = arquivo

    # Adicionar o DataFrame à lista
    df_list.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_combined = pd.concat(df_list, ignore_index=True)

# Ordenar o DataFrame pela coluna 'data'
df_combined['data_lanc'] = pd.to_datetime(df_combined['data_lanc'], errors='coerce', dayfirst=True)  # Converter a coluna 'data' para datetime
df_combined = df_combined.sort_values(by='data_lanc')  # Ordenar pela coluna 'data'

# Adicionar uma coluna 'data_carga' com a data atual
df_combined['data_carga'] = pd.to_datetime('today').normalize()  # 'normalize' para deixar a hora como 00:00:00

colunas_finais = ['data_lanc', 'lancamento', 'ag_origem', 'valor', 'saldos', 'idcategoria', 'observacao','nome_arquivo','data_carga']

df_filtrado = df_combined[colunas_finais]

# Exibir as primeiras linhas do DataFrame combinado com a nova coluna
df_filtrado.head()

# exibir a quantidade de linhas para cada arquivo
# print(df_filtrado.groupby('nome_arquivo').size())


,data_lanc,lancamento,ag_origem,valor,saldos,idcategoria,observacao,nome_arquivo,data_carga
0,2025-04-01,PAY -CAFE 01/04,9128.0,-17.00,NaN,Alim08,NaN,2025-04-Extrato Conta Corrente-260420251441.xlsx,2025-05-10
1,2025-04-01,SALDO DO DIA,NaN,NaN,30.6,NaN,NaN,2025-04-Extrato Conta Corrente-260420251441.xlsx,2025-05-10
2,2025-04-02,IOF,NaN,-0.72,NaN,Imp06,NaN,2025-04-Extrato Conta Corrente-260420251441.xlsx,2025-05-10
3,2025-04-02,RSHOP-DianaCristi-02/04,9773.0,-27.00,NaN,Outr03,NaN,2025-04-Extrato Conta Corrente-260420251441.xlsx,2025-05-10
4,2025-04-02,RSHOP-Seceli -02/04,9120.0,-10.90,NaN,Merc04,NaN,2025-04-Extrato Conta Corrente-260420251441.xlsx,2025-05-10


#### CRIA A CONEXÃO COM O BANCO DE DADOS

In [2]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

# Credenciais do banco de dados
dbname = "neondb"
user = "neondb_owner"
password = "npg_ayeAjRZ8sPD7"
host = "ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech"
port = "5432"

# Criar a string de conexão com o PostgreSQL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Criar a conexão com o banco de dados
engine = create_engine(connection_string)

#### GRAVA OS DADOS NO DATABASE VERIFICANDO SE OS DADOS JÁ EXISTEM OU NÃO NA TABELA

In [3]:
from sqlalchemy import text

# Função para excluir dados antigos e inserir novos, com relatórios
def excluir_e_inserir(df, engine):
    with engine.connect() as conn:
        # Verificar se a tabela existe no banco
        check_table_query = text("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_schema = 'fluxo'
                AND table_name = 'itau'
            );
        """)
        table_exists = conn.execute(check_table_query).scalar()

        if table_exists:
            # Se a tabela existir, buscar os arquivos já importados
            query = "SELECT nome_arquivo FROM fluxo.itau GROUP BY nome_arquivo;"
            arquivos_importados = pd.read_sql(query, engine)
        else:
            # Se a tabela não existir, considera que nenhum arquivo foi importado ainda
            arquivos_importados = pd.DataFrame(columns=['nome_arquivo'])

    # Filtrar os arquivos no df que já foram importados
    arquivos_para_atualizar = df[df['nome_arquivo'].isin(arquivos_importados['nome_arquivo'])]

    # Contar o número de registros a serem excluídos
    num_excluidos = 0
    if not arquivos_para_atualizar.empty:
        with engine.connect() as conn:
            for arquivo in arquivos_para_atualizar['nome_arquivo'].unique():
                delete_query = text(f"DELETE FROM fluxo.itau WHERE nome_arquivo = '{arquivo}';")
                result = conn.execute(delete_query)
                conn.commit()  # Confirmar a exclusão
                num_excluidos += result.rowcount  # Contabiliza as linhas excluídas

    # Contar o número de registros a serem inseridos
    num_incluidos = len(df)

    # Inserir os novos dados ou dados atualizados no banco de dados
    df.to_sql('itau', engine, schema='fluxo', if_exists='append', index=False)

    # Relatório de execução
    print(f"Processamento concluído.")
    print(f"Número de arquivos processados: {df['nome_arquivo'].nunique()}")
    print(f"Número de linhas excluídas: {num_excluidos}")
    print(f"Número de linhas inseridas: {num_incluidos}")

# Chamar a função para excluir dados antigos e inserir os novos
excluir_e_inserir(df_filtrado, engine)


Processamento concluído.
Número de arquivos processados: 2
Número de linhas excluídas: 0
Número de linhas inseridas: 105


#### MOVE OS ARQUIVOS JÁ INSERIDOS NO BANCO PARA UMA OUTRA PASTA

In [4]:
import os
import shutil

def mover_arquivos_processados(folder_path, arquivos):
    """Move os arquivos processados para uma pasta 'Processed' dentro do diretório principal."""
    processed_folder = os.path.join(folder_path, "Processed")

    # Criar a pasta "Processed" se não existir
    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)

    for arquivo in arquivos:
        origem = os.path.join(folder_path, arquivo)
        destino = os.path.join(processed_folder, arquivo)

        # Verifica se o arquivo existe antes de mover
        if os.path.exists(origem):
            shutil.move(origem, destino)
            print(f"Arquivo movido para 'Processed': {arquivo}")
        else:
            print(f"Arquivo não encontrado para mover: {arquivo}")

# Após a inserção no banco, chamar a função para mover os arquivos
mover_arquivos_processados(folder_path, arquivos)


Arquivo movido para 'Processed': 2025-04-Extrato Conta Corrente-260420251441.xlsx
Arquivo movido para 'Processed': 2025-05-Extrato Conta Corrente-030520251107.xlsx
